In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

# Data base import

In [ ]:
from utils import sample_data

# -- Get the data -- #
NB_SAMPLES = 10
PATH_SAMPLE = Path("../data/samples/sample_{}_train.csv".format(NB_SAMPLES))
DATA_SAMPLE = pd.read_csv(PATH_SAMPLE).to_numpy()

In [ ]:
print(DATA_SAMPLE)

# Pre-processing

In [ ]:
from descriptors.glove_descriptor.glove_descriptor import descriptor
from utils.data_descriptor import convert_labels

# -- Parameters -- #
WORD_SIZE = 50  # 50 or 100 or 200 or 300
SENTENCE_SIZE = 50  # What ever
FILL_WITH = 0  # If a word is not in the dictionary, [0, ..., 0] will describe it.
SENTIMENT_WEIGHT = 2  # Multiply the sentiment by a factor
OPTIONS = [WORD_SIZE, SENTENCE_SIZE, FILL_WITH, SENTIMENT_WEIGHT]

SPLIT_PUNCTUATION = False # to tell wheter the puncutation "!?.;,/" etc are keeped sticked to a word or not


# -- Get the descriptor -- #
PATH_DICTIONARY = Path("../data/glove_descriptor/glove.6B.{}d.txt".format(WORD_SIZE))
DICTIONARY = pd.read_csv(PATH_DICTIONARY, sep=" ", header=None)


# -- Describe every tweet -- #
(TWEET_STRING, TWEET_SCALAR) = descriptor(DATA_SAMPLE, DICTIONARY, OPTIONS)

# -- Get the labels -- #
IMPORTANT_WORDS = convert_labels(DATA_SAMPLE, SENTENCE_SIZE)

# -- Get the original tweets -- #
TWEET_ORIGINAL = DATA_SAMPLE[:, 1]

print("\nString tweet :")
print(TWEET_STRING[0], "\nlen(TWEET_STRING) :", len(TWEET_STRING))
print("\nDescriptor :")
print(TWEET_SCALAR[0], "\nTWEET_SCALAR.shape :", TWEET_SCALAR.shape)
print("\nLabel :")
print(IMPORTANT_WORDS[0], "\nIMPORTANT_WORDS.shape :", IMPORTANT_WORDS.shape)
print("\nOriginal tweet")
print(TWEET_ORIGINAL[0], "\nlen(TWEET_ORIGINAL) :", len(TWEET_ORIGINAL))

# Classification

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
nb_neighbors = 1

knn = KNeighborsRegressor(nb_neighbors, weights="distance")
knn.fit(TWEET_SCALAR, IMPORTANT_WORDS)

In [ ]:
PREDICTIONS = knn.predict(TWEET_SCALAR)

In [ ]:
from utils.post_processing import pred_to_string

RESULTS = np.zeros(len(PREDICTIONS), dtype=object)
for i in range(len(PREDICTIONS)):
    RESULTS[i] = pred_to_string(TWEET_ORIGINAL[i], TWEET_STRING[i], PREDICTIONS[i])

In [ ]:
for i in range(len(results)):
    print(DATA[i, 1])
    print(DATA[i, 2])
    print(RESULTS[i])
    print()

In [ ]:
from utils.loss import jaccard

AVG = 0
for i in range(len(RESULTS)):
    AVG += jaccard(RESULTS[i], DATA[i, 2])
    
    if jaccard(RESULTS[i], DATA_SAMPLE[i, 2]) != 1:
        print("Error on", i)
AVG /= len(RESULTS)

print(AVG)